In [401]:
import pandas as pd
import numpy as np
from random import randint

## acquire data
train_df = pd.read_csv('train.csv')
gender_df = pd.read_csv('gender_submission.csv')
test_df = pd.read_csv('test.csv')

#print(train_df.info())
#print(test_df.info())
#print(gender_df.info())

In [402]:
## check what data is impactful

## try Embarked
## there are some Embarked values missing, add with the most frequent
embarked_fill = train_df['Embarked'].mode()
#print(embarked_fill)

embarked_null = train_df['Embarked'].isnull().sum()

for b in range (0,embarked_null):
    train_df['Embarked'].fillna(embarked_null, inplace = True)

train = (train_df[['Embarked','Survived']].groupby(["Embarked"]).mean())

#print(train_df.info())
#print(train)

In [403]:
## try Pclass
train = (train_df[['Pclass','Survived']].groupby(["Pclass"]).mean())
## findings to be impactful: Pclass

In [404]:
## try Gender
train = (train_df[['Sex','Survived']].groupby(["Sex"]).mean())
## findings to be impactful: Pclass, Sex, Embarked

In [405]:
## try Age

## age has several data missing, need to fill
## null age + mean age + std deviation age // or use interpolate()
not_null_age = train_df['Age'].count()
null_age = train_df['Age'].isnull().sum()

std_age = train_df['Age'].std()
avg_age = train_df['Age'].mean()
summation = int(avg_age+std_age)
diff = int(avg_age-std_age)

for a in range(0,null_age):
    rand_age = randint(diff,summation)
    #rand_age = train_df['Age'].interpolate()
    train_df['Age'].fillna(rand_age,inplace = True)
    test_df['Age'].fillna(rand_age,inplace=True)

   
train_df['age_map'] = pd.cut(train_df['Age'],4,labels= [3,2,1,0])

test_df['age_map'] = pd.cut(test_df['Age'],4,labels=[3,2,1,0])
train = (train_df[['Age','Survived']].groupby([cat_age]).mean()) 
print(train_df.info())
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       891 non-null object
age_map        891 non-null category
dtypes: category(1), float64(2), int64(5), object(5)
memory usage: 84.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket       

In [406]:
## findings to be impactful: Pclass, Gender, Embarked, Age
# now mapping the data. turn everything to numbers
train_df['Sex'] = train_df['Sex'].replace({'female': 1, 'male': 0})
test_df['Sex'] = test_df['Sex'].replace({'female':1,'male':0})
train_df['Embarked'] = train_df['Embarked'].replace({'S':0,'Q':1,'C':2})
test_df['Embarked'] = test_df['Embarked'].replace({'S':0,'Q':1,'C':2})
#cat_age = age map
train_df['Age']=train_df['age_map']
test_df['Age'] = test_df['age_map']

#list of columns to drop; 
drop_columns = ['Name','SibSp', 'Parch', 'Fare', 'Cabin', 'Ticket']
train_df = train_df.drop(drop_columns,axis=1)
train_df = train_df.drop('PassengerId',axis=1)
train_df = train_df.drop('age_map',axis=1)
test_df = test_df.drop(drop_columns,axis=1)
test_df = test_df.drop('age_map',axis=1)
print(test_df.head(5))


   PassengerId  Pclass  Sex Age  Embarked
0          892       3    0   2         1
1          893       3    1   1         0
2          894       2    0   0         1
3          895       3    0   2         0
4          896       3    1   2         0


In [407]:
train_X = train_df.drop('Survived',axis=1)
train_Y = train_df['Survived']
test_X = test_df.drop("PassengerId", axis=1)
print(test_X.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
Pclass      418 non-null int64
Sex         418 non-null int64
Age         418 non-null category
Embarked    418 non-null int64
dtypes: category(1), int64(3)
memory usage: 10.5 KB
None


In [408]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier()
decision_tree.fit(train_X,train_Y)
pred_Y = decision_tree.predict(test_X)
accuracy = round(decision_tree.score(train_X,train_Y)*100,2)
print("Accuracy: ",accuracy)

Accuracy:  81.59
